In [6]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage
from langchain_core.messages import ToolMessage
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
import os
import requests


#region Initialization of LLM and CMS
def initialize_config():
    llm = ChatOllama(base_url='http://localhost:11434', model="mistral")
    load_dotenv()
    cms_config = {
        "base_url": os.getenv("CONTENTSTACK_URL"),
        "api_key": os.getenv("CONTENTSTACK_API_KEY"),
        "delivery_token": os.getenv("CONTENTSTACK_DELIVERY_TOKEN"),
        "management_token": os.getenv("CONTENTSTACK_MANAGEMENT_TOKEN"),
        "environment": os.getenv("CONTENTSTACK_ENVIRONMENT")
    }

    config_variables = {"llm": llm, "cms": cms_config}
    print("Connected CMS: " + cms_config["base_url"])
    print("Connected LLM: " + llm.model)

    return config_variables
#endregion

# Initialize LLM and CMS configuration
config = initialize_config()

#region agent tools
@tool
def personalize_informational_text(pageuid:str, generated_content:str):
    """Personalize the information of a product in ContentStack"""

    #state['block_index_to_customize'] = 0 # Set block index to two (second block will be edited)

    headers = {
        "api_key": config["cms"]["api_key"],
        "authorization": config["cms"]["management_token"],
        "Content-Type": "application/json"
    }

    body = {
        "entry": {
            "blocks": {
                "UPDATE": {
                    "index": 0,
                    "data": {
                        "block": {
                            "copy": generated_content,
                        }
                    }
                }
            }
        }
    }

    url = f"https://{config['cms']['base_url']}/v3/content_types/page/entries/{pageuid}"
    print("API " + url + f" contacted with stack {config['cms']['api_key']}.")

    try:
        response = requests.put(url, headers=headers, json=body)
        response.raise_for_status()
        response = response.json()

        return response
    except requests.exceptions.RequestException as e:
        return f"An error occured when updating pages: {str(e)}"

tools = [personalize_informational_text]
#endregion

#region Langgraph nodes
class AgentState(TypedDict):
    messages: list
    pages_list: dict
    content_type_uid: str
    entry_to_customize: str
    block_index_to_customize: int

def retrieve_pages_node(state: AgentState):
    """Retrieve all existing pages from ContentStack"""
    headers = {
        "api_key": config["cms"]["api_key"],
        "access_token": config["cms"]["delivery_token"],
        "Content-Type": "application/json"
    }

    params = {
        "environment": config["cms"]["environment"]
    }

    url = f"https://{config['cms']['base_url']}/v3/content_types/{state['content_type_uid']}/entries"
    print("API " + url + f" contacted with stack {config['cms']['api_key']}.")
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        page_list = response.json()

        state['pages_list'] = page_list
        return state
    except requests.exceptions.RequestException as e:
        return f"Error retrieving pages: {str(e)}"

def personalize_page(state: AgentState):
    """Personalize a page"""

    # receive uid of page to be customized (i.e. homepage). For now this is fixed for the informational text on the homepage.
    agent = create_react_agent(config["llm"], tools)

    # Generate informational text with agent
    try:
        result = agent.invoke({
            "messages": [("user", f"""Personalize the landing page by generating a personalized informational text. There are a few steps you have to follow:
            1. Find the landing page in the pages list and look at the second block contents and remember the uid of the page
            2. Generate a well-suited informational text based on the content of the second block and the user's interests
            3. Use this uid together with your tools to change the information of the product in ContentStack.

            Information for you:
            User interests: The user works at a construction company.
            Pages list: {state['pages_list']}
            """)]})
    except requests.exceptions.RequestException as e:
        return f"An error occured when generating the informational text: {str(e)}"

    # customize the info text

    # TODO: dynamically personalize components:
    # personalize header
    # personalize informational text tailored to industry
    # personalize recommendations (what page most relevant for person) --> Example Bibby: Construction worker should have construction as first option.
    return state
#endregion

# Create graph
graph = StateGraph(AgentState)
graph.add_node("RetrievePages", retrieve_pages_node)
graph.add_node("personalizePage", personalize_page)
graph.add_edge(START, "RetrievePages")
graph.add_edge("RetrievePages", "personalizePage")
graph.add_edge("personalizePage", END)


retrievalAgent = graph.compile()

from IPython.display import Image, display
display(Image(retrievalAgent.get_graph().draw_mermaid_png()))

'''
# Run agent
result = retrievalAgent.invoke({"content_type_uid": "page"})
print(result["pages_list"])'''











Connected CMS: azure-eu-api.contentstack.com
Connected LLM: mistral


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`